In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

### Goal: determine if it's even worth it to fit bikes into categories or is it more or less a continuous spectrum

In [11]:
data = pd.read_csv('mtb_data.csv')

In [12]:
data.drop(columns=['crank_length','f_piston','r_piston', 'size_used'], inplace=True)

In [14]:
data = data.fillna(data.mean())
data.head(30)

,model,brand,setting,label,rear_travel,fork_travel,f_rotor_dim,r_rotor_dim,head_angle,seat_angle,stem_length,handlbar_width,reach,stack,wheelbase,chainstay_length,bb_height,standover_height
0,epic,specialized,na,xc,100,100,180,160,67.5,75.5,70.0000,760.000000,445.000000,591.000000,1148.0,433.0,324.000000,766.000000
1,epic evo,specialized,high,dc,110,120,180,160,67.0,74.5,60.0000,760.000000,436.000000,597.000000,1164.0,438.0,339.000000,781.000000
2,epic evo,specialized,low,dc,110,120,180,160,66.5,74.5,60.0000,760.000000,436.000000,597.000000,1164.0,438.0,336.000000,781.000000
3,stumpjumper,specialized,high,tr,130,140,200,180,65.5,76.5,50.0000,780.000000,480.000000,627.000000,1228.0,432.0,340.000000,749.000000
4,stumpjumper,specialized,low,tr,130,140,200,180,65.0,76.0,50.0000,780.000000,475.000000,632.000000,1228.0,432.0,333.000000,749.000000
5,stumpjumper evo,specialized,na,tr,150,160,200,200,63.0,76.9,50.0000,800.000000,475.000000,635.000000,1247.0,438.0,340.000000,762.000000
6,enduro,specialized,high,en,170,170,220,200,64.3,76.0,50.0000,800.000000,458.087179,612.994872,1274.0,442.0,338.493103,745.568421
7,enduro,specialized,low,en,170,170,220,200,63.9,76.0,50.0000,800.000000,487.000000,629.000000,1274.0,442.0,347.000000,745.568421
8,supercaliber,trek,na,xc,60,100,160,160,69.0,74.0,70.0000,720.000000,440.000000,594.000000,1121.0,430.0,320.000000,760.000000
9,slash,trek,high,en,160,170,203,203,64.6,76.1,50.0000,820.000000,474.000000,622.000000,1242.0,435.0,352.000000,749.000000


In [14]:
# rows = data.shape[0]
# print('Total rows:', rows)

# #impute missing data with average
# newdata = {}
# for col in data.columns:
#     newdata[col] = []
# newdf = pd.DataFrame(newdata)

# print(newdf)
# for cat in data['label'].unique():
#     cat_df = data[data['label'] == cat]
#     for col in cat_df.columns[5:]:
#         mean = cat_df[col].mean(skipna=True)
        
#         cat_df[col] = cat_df[col].fillna(mean)
#     newdf = pd.concat([newdf, cat_df])
    
# newdf.to_csv('edited_data.csv')

Total rows: 40
Empty DataFrame
Columns: [model, brand, setting, label, rear_travel, fork_travel, f_rotor_dim, r_rotor_dim, head_angle, seat_angle, stem_length, handlbar_width, reach, stack, wheelbase, chainstay_length, bb_height, standover_height]
Index: []


<ipython-input-14-3a8c23659cb2>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_df[col] = cat_df[col].fillna(mean)


In [21]:
# limit to enduro or xc for right now
newdf = data.copy()
newdf = newdf[newdf['label'].isin(['en',  'xc'])]
newdf

,model,brand,setting,label,rear_travel,fork_travel,f_rotor_dim,r_rotor_dim,head_angle,seat_angle,stem_length,handlbar_width,reach,stack,wheelbase,chainstay_length,bb_height,standover_height
0,epic,specialized,na,xc,100,100,180,160,67.5,75.5,70.0000,760.000000,445.000000,591.000000,1148.0,433.0,324.000000,766.000000
6,enduro,specialized,high,en,170,170,220,200,64.3,76.0,50.0000,800.000000,458.087179,612.994872,1274.0,442.0,338.493103,745.568421
7,enduro,specialized,low,en,170,170,220,200,63.9,76.0,50.0000,800.000000,487.000000,629.000000,1274.0,442.0,347.000000,745.568421
8,supercaliber,trek,na,xc,60,100,160,160,69.0,74.0,70.0000,720.000000,440.000000,594.000000,1121.0,430.0,320.000000,760.000000
9,slash,trek,high,en,160,170,203,203,64.6,76.1,50.0000,820.000000,474.000000,622.000000,1242.0,435.0,352.000000,749.000000
10,slash,trek,low,en,160,170,203,203,64.1,75.6,50.0000,820.000000,469.000000,626.000000,1243.0,437.0,345.000000,743.000000
15,spark rc,scott,NaN,xc,100,110,180,160,68.5,73.8,80.0000,740.000000,456.800000,596.200000,1158.6,435.0,319.500000,756.000000
19,ransom,scott,NaN,en,170,170,203,180,64.5,75.0,50.0000,800.000000,466.500000,627.600000,1249.2,437.9,353.000000,760.900000
20,anthem advanced pro,giant,NaN,xc,90,100,180,160,69.0,73.5,80.0000,780.000000,454.000000,594.000000,1154.0,438.0,338.493103,817.000000
23,reign advanced pro,giant,NaN,en,146,170,203,203,64.6,76.4,40.0000,800.000000,488.000000,631.000000,1262.0,439.0,338.493103,781.000000


In [16]:
## average high/low rows so as not to double emphasize the same bike w/ two different data points


#is this necessary???
#TODO maintain labels after averaging
newdf['setting'] = newdf['setting'].fillna('na')
settings_df = newdf[newdf['setting'] != 'na']
na_settings_df = newdf[newdf['setting'] == 'na']

settings_df = settings_df.drop(columns=['setting'])
na_settings_df = na_settings_df.drop(columns=['setting'])

settings_df = settings_df.groupby(['model', 'brand']).mean()

avgd_df = pd.concat([settings_df, na_settings_df])

print(avgd_df.head())
print(avgd_df.shape)

TypeError: list indices must be integers or slices, not str

In [24]:
avgd_df = newdf.copy()
avgd_df.head()

,model,brand,setting,label,rear_travel,fork_travel,f_rotor_dim,r_rotor_dim,head_angle,seat_angle,stem_length,handlbar_width,reach,stack,wheelbase,chainstay_length,bb_height,standover_height
0,epic,specialized,na,xc,100,100,180,160,67.5,75.5,70.0,760.0,445.000000,591.000000,1148.0,433.0,324.000000,766.000000
6,enduro,specialized,high,en,170,170,220,200,64.3,76.0,50.0,800.0,458.087179,612.994872,1274.0,442.0,338.493103,745.568421
7,enduro,specialized,low,en,170,170,220,200,63.9,76.0,50.0,800.0,487.000000,629.000000,1274.0,442.0,347.000000,745.568421
8,supercaliber,trek,na,xc,60,100,160,160,69.0,74.0,70.0,720.0,440.000000,594.000000,1121.0,430.0,320.000000,760.000000
9,slash,trek,high,en,160,170,203,203,64.6,76.1,50.0,820.0,474.000000,622.000000,1242.0,435.0,352.000000,749.000000


In [28]:
#numerical data
labels = avgd_df['label']
num_data = avgd_df.drop(columns=['model', 'brand', 'setting', 'label'])
num_mtx = num_data.to_numpy()

In [26]:
print(labels)

0     xc
6     en
7     en
8     xc
9     en
10    en
15    xc
19    en
20    xc
23    en
24    xc
27    en
30    en
31    xc
39    xc
Name: label, dtype: object


In [29]:
# Scaling data
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(num_data), columns=num_data.columns)
num_mtx = df_scaled.to_numpy()

In [30]:
#project on 2 axes
pca = PCA(n_components=2)
pca.fit(num_mtx)
axes = pca.components_.T
print(pca.explained_variance_ratio_)

print('data shape')
print(num_mtx.shape)

print('axes shape')
print(axes.shape)

proj_data = num_mtx@axes

print('projected data shape')
print(proj_data.shape)

[0.78883773 0.06500068]
data shape
(15, 14)
axes shape
(14, 2)
projected data shape
(15, 2)


In [32]:
def map_color(l):
    if l == 'xc':
        return 'b'
    elif l == 'dc':
        return 'g'
    elif l == 'tr':
        return 'p'
    else:
        return 'r'

    
labels['color'] = labels.apply(lambda x: map_color(x))

plt.scatter(proj_data[:,0], proj_data[:,1], c = labels['color'])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().